In [47]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pickle
from grakel.kernels import GraphletSampling
from sklearn.svm import SVC
import numpy as np
import networkx as nx
import torch
from communityaware.perturb import _perturb_graph_vmap
import grakel
from timebudget import timebudget

In [56]:
P = (0.1, 0.45)
repeats = 1000

In [57]:
# value of p
noise_matrix = np.zeros((10, 10))
noise_matrix[0, 2] = P[0]
noise_matrix[0, 3] = P[0]
noise_matrix[1, 3] = P[0]
noise_matrix[1, 4] = P[0]
noise_matrix[2, 4] = P[0]
noise_matrix = noise_matrix + noise_matrix.T
noise_matrix[5:, 5:] = P[1]
np.fill_diagonal(noise_matrix, 0)
noise_matrix = torch.tensor(noise_matrix)

In [58]:
# load data
data = pickle.load(open('../data/kernel/data.pkl', 'rb'))
G_train = data['G_train']
y_train = data['y_train']
G_test = data['G_test']
y_test = data['y_test']

# fit the kernel
kernel = GraphletSampling(k=5)
K_train = kernel.fit_transform(G_train)
clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train)

SVC(kernel='precomputed')

In [59]:
votes = []
for test_sample, label in zip(G_test, y_test):
    
    # extract the graph
    graph_dict = test_sample[0]
    graph = nx.Graph()
    graph.add_nodes_from(sorted(graph_dict.keys()))
    for node in graph_dict:
        for neighbour in graph_dict[node]:
            graph.add_edge(node, neighbour)
    A = torch.tensor(np.array(nx.adjacency_matrix(graph).todense()))

    # perturb graphs
    perturbed_graphs = _perturb_graph_vmap(A.unsqueeze(0).repeat(repeats, 1, 1), noise_matrix.unsqueeze(0).repeat(repeats, 1, 1)).numpy()
    perturbed_graphs = [nx.from_numpy_matrix(perturbed_graph) for perturbed_graph in perturbed_graphs]
    perturbed_graphs = grakel.graph_from_networkx(perturbed_graphs)

    with timebudget('Make test kernel'):
        K_test = kernel.transform(perturbed_graphs)

    
    y_pred = clf.predict(K_test)
    test_accuracy = np.mean(y_pred == label)

    print(test_accuracy)
    votes.append(torch.bincount(torch.tensor(y_pred).transpose(-1, 0), minlength=2))
    break

Make test kernel took 15.956sec
0.89


In [63]:
from os.path import join


fname

'0.1_0.45'

In [55]:
len(G_test)

50

In [65]:
torch.load('../output/kernel/votes/0.05_0.45')

tensor([[10,  0],
        [ 1,  9],
        [10,  0],
        [ 2,  8],
        [ 1,  9],
        [10,  0],
        [ 0, 10],
        [10,  0],
        [10,  0],
        [ 0, 10]])

In [66]:
# load data
data = pickle.load(open('../data/kernel/data.pkl', 'rb'))
G_train = data['G_train']
y_train = data['y_train']
G_test = data['G_test']
y_test = data['y_test']


In [67]:
y_test

array([0, 1, 0, 1, 1, 0, 1, 0, 0, 1])